In [50]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt

# Ingest data and cleanup 
properties_data = []

## Read the file line by line
with open('MixPanel_export-1691969806891.txt', 'r') as f:
    for line in f:
        # Load the JSON data from each line
        json_line = json.loads(line)
        # Add the JSON data to the list
        properties_data.append(json_line)

df = pd.json_normalize(properties_data)

## Select the required columns
df = df[['properties.sellAmountUsd', 'properties.sellAssetId', 'properties.buyAssetId', 'properties.quoteMeta', 'properties.version', 'properties.distinct_id']]

## Cleanup the nested headers
df = df.rename(columns=lambda x: x.replace('properties.', ''))

## Use the latest data schema and bug fixes
df = df.query('version == "20230823"')

## Filter duplicate quotes with the same input args from the same user (i.e. polling qoute updates)
columns_to_check = ['sellAmountUsd', 'sellAssetId', 'buyAssetId', 'distinct_id']
df = df.drop_duplicates(subset=columns_to_check, keep='first')

## Handle current inputOutputRatio = 0, which returns a differenceFromBestQuoteDecimalPercentage of "1", by removing these qoutes (usually happens when the sell amount is too low).
df['quoteMeta'] = df['quoteMeta'].apply(lambda x: [item for item in x if item['differenceFromBestQuoteDecimalPercentage'] != "1"])

## Create a new DataFrame containing only EVM trades, with filtered rows where sellAssetId and buyAssetId start with eip155
df_evm = df.loc[(df['sellAssetId'].str.startswith('eip155')) & (df['buyAssetId'].str.startswith('eip155'))].copy()


## Extract prefix before ":" and compare for sellAssetId and buyAssetId in the new DataFrame
df_evm.loc[:, 'sellAssetPrefix'] = df_evm['sellAssetId'].str.split('/').str[0]
df_evm.loc[:, 'buyAssetPrefix'] = df_evm['buyAssetId'].str.split('/').str[0]

df_evm_same_chain = df_evm[df_evm['sellAssetPrefix'] == df_evm['buyAssetPrefix']]


## Log the DF schema as a sanity check
df_evm_same_chain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 351 entries, 30 to 2491
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sellAmountUsd    351 non-null    object
 1   sellAssetId      351 non-null    object
 2   buyAssetId       351 non-null    object
 3   quoteMeta        351 non-null    object
 4   version          351 non-null    object
 5   distinct_id      351 non-null    object
 6   sellAssetPrefix  351 non-null    object
 7   buyAssetPrefix   351 non-null    object
dtypes: object(8)
memory usage: 24.7+ KB


In [51]:
# Log a row for reference

# Get a row of the DataFrame as a dict
first_row_dict = df_evm_same_chain.iloc[30].to_dict()

# Convert the dict to a formatted JSON string
formatted_json = json.dumps(first_row_dict, indent=4)

# Print the formatted JSON
print(formatted_json)

{
    "sellAmountUsd": "184786",
    "sellAssetId": "eip155:1/slip44:60",
    "buyAssetId": "eip155:1/erc20:0xc770eefad204b5180df6a14ee197d99d808ee52d",
    "quoteMeta": [
        {
            "differenceFromBestQuoteDecimalPercentage": 0,
            "quoteReceived": true,
            "swapperName": "0x"
        },
        {
            "differenceFromBestQuoteDecimalPercentage": 0.005013372032184327,
            "quoteReceived": true,
            "swapperName": "LI.FI"
        },
        {
            "differenceFromBestQuoteDecimalPercentage": 0.6444536762200204,
            "quoteReceived": true,
            "swapperName": "THORChain"
        },
        {
            "differenceFromBestQuoteDecimalPercentage": null,
            "quoteReceived": false,
            "swapperName": "1INCH"
        },
        {
            "differenceFromBestQuoteDecimalPercentage": null,
            "quoteReceived": false,
            "swapperName": "CoW Swap"
        },
        {
            "differe

In [53]:
# When we got a quote via 0x, and another swapper gave us a quote - what % of the time was 0x the best (for EVM, non-bridging quotes)?

## Determine the indices where there is at least one other swapper's quote received
indices_with_other_swappers = df_evm_same_chain[df_evm_same_chain['quoteMeta'].apply(lambda x: any(q['swapperName'] != '0x' and q['quoteReceived'] == True for q in x))].index

## Now use the original DataFrame to find only rows where 'swapperName' is '0x' and that meet the condition above
quote_meta_data_with_other_swappers = pd.json_normalize(df_evm_same_chain.loc[indices_with_other_swappers].explode('quoteMeta').reset_index().quoteMeta)

## Group by swapper name and calculate the percentage where 'differenceFromBestQuoteDecimalPercentage' equals 0 (meaning it is the best swapper) for each group
percentage_best_by_swapper = quote_meta_data_with_other_swappers.groupby('swapperName').apply(
    lambda group: (group['differenceFromBestQuoteDecimalPercentage'] == 0).mean() * 100
).sort_values(ascending=False)

## Concatenate the results into a string
output_swappers = ", ".join([
    f"{swapper}: {percentage:.2f}%" for swapper, percentage in percentage_best_by_swapper.items()
])

output_swappers

# Create a DataFrame from the sorted percentages
swapper_table = pd.DataFrame(percentage_best_by_swapper_sorted, columns=['Percentage'])
swapper_table['Percentage'] = swapper_table['Percentage'].apply(lambda x: f"{x:.2f}%")

# Optionally print to console
print(swapper_table)


NameError: name 'percentage_best_by_swapper_sorted' is not defined